## Introduction

This analysis seeks to gather and prove valuable observations about the impact of various home attributes on its value. The project combines multiple publicly available datasets [made available](https://data.kingcounty.gov/) by King County. One dataset provides a record of home and land sales alongside various identifying feautres. The second dataset gives very granular data about housing in King County, going beyond just square footage to a breakdown by room, many identifying features such as porch size, and more. Combining these two datasets opens the door for the analysis to come. 

This notebook provides a somewhat condensed analysis compared to the full sequence necessary to understand the full details of choosing specific models and the nitty-gritty details. Please refer to the notebooks in the repository folder notebooks->exploratory if you would like to see a deep dive.

### The Process
1. Basic Setup and Data Assembly
2. Data Aggregation and Cleaning
3. Feature Selection and Creation
4. The Model
5. Quantify The Findings

## 1. Basic Setup and Data Assembly

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import sqlite3
import os, sys
import warnings

# Import functions from a Python file in this repository with context-relevant functionality
path_to_src = os.path.join('..', '..', 'src')
sys.path.insert(1, path_to_src)
from custom_functions import *
pd.set_option('display.max_columns', 100)
%load_ext autoreload
%autoreload 2

##### Import King County housing data

In [2]:
# Read csv files from the data directory
df_lookup = pd.read_csv(os.path.join('..','..', 'data', 'raw', 'EXTR_LookUp.csv'), dtype='str')
df_resbldg = pd.read_csv(os.path.join('..','..', 'data', 'raw', 'EXTR_ResBldg.csv'), dtype='str')
df_rpsale = pd.read_csv(os.path.join('..','..', 'data', 'raw', 'EXTR_RpSale.csv'), dtype='str')

# Use the user-defined strip_spaces function to remove leading and trailing spaces from the entire dataframe
df_lookup = strip_spaces(df_lookup)
df_resbldg = strip_spaces(df_resbldg)
df_rpsale = strip_spaces(df_rpsale)

## 2. Data Aggregation and Cleaning

### The Sales Database

##### Eliminate unecessary data. After close investigation, the below columns were deemed the most worthy of continued analysis.

In [3]:
# Manual selection of the features of choice
rpsale_desired_columns = ['ExciseTaxNbr', 'Major', 'Minor', 'DocumentDate', 'SalePrice', 'RecordingNbr', 'PropertyType', 
                          'PrincipalUse', 'SaleInstrument', 'AFForestLand', 'AFCurrentUseLand', 'AFNonProfitUse', 
                          'AFHistoricProperty', 'SaleReason', 'PropertyClass', 'SaleWarning']

# Remove all columns that are not in the above lists
df_rpsale = df_rpsale[rpsale_desired_columns].copy()

##### Create identifier that will be used to connect the two dataframes. 
In this case, each database provides *Major* and *Minor*, which serve as location-specific identifiers. From here on, the combination of *Major* and *Minor* will simply be referred to as the *parcel*. Although there is often more than one sale associated with a parcel, this is a great place to start for narrowing down our search. The goal is to narrow down the *Sales* dataset to include only one sale per parcel. This allows for a connection with the second database, *Residential Buildings*. 

In [4]:
# Create ParcelID
df_rpsale['Parcel_ID'] = df_rpsale.Major + '-' + df_rpsale.Minor

#### The Sales Database
Starting with some of the nitty gritty data selection

In [5]:
# Select only sales for "Residential" plots, corresponding to code #6, as can be found in the data dictionary
# This eliminates Commerical, Condominium, Apartment, etc.
df_rpsale['PrincipalUse'] = elimination_by_code(df_rpsale['PrincipalUse'], '6')

# PropertyClass is another distinction between Commerical/Industrial and Residential, as well as 
# other fundamental features. Code #8 corresponds to Residential Improved property
df_rpsale['PropertyClass'] = elimination_by_code(df_rpsale['PropertyClass'], '8')

# Yet another classification of property type. Code #11 corresponds to single family households
# Here we eliminate multiple family residences, alongside many commercial uses
df_rpsale['PropertyType'] = elimination_by_code(df_rpsale['PropertyType'], '11')
df_rpsale.dropna(inplace=True)

##### Limit scope to 2019 sales

In [6]:
# Type conversion
df_rpsale['DocumentDate'] = df_rpsale.DocumentDate.astype(np.datetime64)

# Isolate SaleYear as its own column
df_rpsale['SaleYear'] = [sale.year for sale in df_rpsale['DocumentDate']]

# Eliminate rows corresponding to sales in a year other than 2019
df_rpsale = df_rpsale.loc[df_rpsale['SaleYear']==2019].copy()

##### Eliminate unrealistically small sales

In [7]:
min_acceptable_sale_price = 25000
df_rpsale['SalePrice'] = df_rpsale.SalePrice.astype('int')
df_rpsale = df_rpsale.loc[df_rpsale.SalePrice > min_acceptable_sale_price].copy()

##### Create column to identify duplicates, a necessary process before combining the two datasets

In [8]:
df_rpsale['SaleCount'] = list(map(dict(df_rpsale.Parcel_ID.value_counts()).get, df_rpsale.Parcel_ID))

##### Upon further inspection, there are still duplicates (cases of more than one sale of a given parcel in 2019)

In [9]:
print(df_rpsale.SaleCount.value_counts())

1    17884
2      682
6        6
3        6
Name: SaleCount, dtype: int64


##### Eliminate sales that are not the most recent
This eliminates the duplicated data issue of multiple sales in one year, bringing the dataset one step closer to a unique Parcel ID

In [ ]:
# User-defined function that returns True/False indicating whether a sale is the most recent for each parcel
true_false = identify_latest_sale(df_rpsale.DocumentDate, df_rpsale.Parcel_ID)
df_rpsale = df_rpsale.loc[true_false].copy()

# Recalculate 'SaleCount' after removing old sales as described above
df_rpsale['SaleCount'] = list(map(dict(df_rpsale.Parcel_ID.value_counts()).get, df_rpsale.Parcel_ID))
print(df_rpsale.SaleCount.value_counts())

##### The great majority of parcels are narrowed to one sale. For the remainders, take a simple approach: average the SalePrice for all sales on that parcel. 
Further inspection was done to verify that this is a valid way of dealing with outliers. For example, in many cases, the sales are of equal or nearly equal price. 

In [ ]:
# User-defined function to return the average SalePrice for each parcel with more than one sale
df_rpsale = avg_price_for_duped_parcels(df_rpsale)
df_rpsale['SaleCount'] = list(map(dict(df_rpsale.Parcel_ID.value_counts()).get, df_rpsale.Parcel_ID))


# Remove duplicates
df_rpsale.index = df_rpsale.Parcel_ID.values
df_rpsale = df_rpsale.drop_duplicates('Parcel_ID')

# Verify no duplicates remain
print(df_rpsale.SaleCount.value_counts())

##### As a final step to cleaning *Sales* dataset, eliminate columns that are of no use going forward

In [ ]:
# Drop unused columns as final step of cleaning before join
cols_to_keep = ['SalePrice', 'Parcel_ID', 'PropertyType', 'PrincipalUse', 
                'SaleInstrument', 'AFForestLand', 'AFCurrentUseLand', 
                'AFNonProfitUse', 'AFHistoricProperty', 'SaleReason', 'PropertyClass', 'SaleWarning']
df_rpsale = df_rpsale[cols_to_keep].copy()
df_rpsale.head(5)

### The Buildings Database
This dataset requires far less cleaning than *Sales* did. In this case, it is mostly eliminating columns that have been determined not valuable as well as converting to correct datatypes.

##### Eliminate unecessary data. After close investigation, the below columns were deemed the most worthy of continued analysis.

In [ ]:
# Manual selection of the features of choice
resbldg_desired_columns = ['Major', 'Minor', 'NbrLivingUnits', 'Stories', 'BldgGrade', 
                           'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
                           'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving', 'SqFtTotBasement', 
                           'SqFtFinBasement', 'FinBasementGrade', 'SqFtGarageBasement', 'SqFtGarageAttached', 
                           'DaylightBasement','SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
                           'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms','BathHalfCount', 
                           'Bath3qtrCount', 'BathFullCount', 'FpSingleStory','FpMultiStory', 'FpFreestanding', 
                           'FpAdditional', 'YrBuilt','YrRenovated', 'PcntComplete', 'Obsolescence', 
                           'PcntNetCondition','Condition']


# Remove all columns that are not in one of the above two lists.
df_resbldg = df_resbldg[resbldg_desired_columns].copy()

##### Create Parcel ID, which will map to Sales database 

In [ ]:
# ParcelID is an aggregation of Major and Minor, as it was with the Sales database
df_resbldg['Parcel_ID'] = df_resbldg.Major + '-' + df_resbldg.Minor

##### Convert features to the appropriate data type

In [ ]:
# Convert each of these columns into integer format
convert_to_int = ['SqFtOpenPorch', 'SqFtEnclosedPorch', 'Bedrooms', 'SqFtGarageAttached', 'SqFtGarageBasement', 'NbrLivingUnits', 'BldgGrade', 'SqFt1stFloor',
                  'SqFtHalfFloor', 'SqFt2ndFloor', 'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf','SqFtTotLiving', 'SqFtTotBasement', 'SqFtFinBasement', 
                  'FinBasementGrade', 'SqFtGarageBasement', 'SqFtGarageAttached', 'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'BathHalfCount', 
                  'Bath3qtrCount', 'BathFullCount', 'FpSingleStory', 'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt', 'YrRenovated', 'BrickStone']

# Loop over entire list
for category in convert_to_int:
    df_resbldg[category] = df_resbldg[category].astype('int')
    
    
# Convert into float format
df_resbldg['Stories'] = df_resbldg['Stories'].astype('float')

# Nit-picky
df_resbldg['DaylightBasement'] = df_resbldg['DaylightBasement'].str.upper() # Data cleaning for inconsistent casing
df_resbldg = df_resbldg.loc[df_resbldg.PcntComplete.astype('str') == '0'].copy() # Remove buildings that aren't complete
df_resbldg = df_resbldg.loc[df_resbldg.Obsolescence.astype('str') == '0'].copy() # Remove buildings in obsolescence process
df_resbldg = df_resbldg.loc[df_resbldg.PcntNetCondition.astype('str') == '0'].copy() # Remove 6 outliers in abnormal condition

### Combine *Sales* and *Buildings* datasets
At long last: conduct join using SQL

In [ ]:
# Create a connection with a SQL DataBase file
path_to_db = os.path.join('..', '..', 'data', 'main.db')
connection = sqlite3.connect(path_to_db)
df_resbldg.to_sql('buildings', connection, if_exists='replace')
df_rpsale.to_sql('sales', connection, if_exists='replace')

# Join Sales and Buildings data together
query = '''SELECT * FROM buildings LEFT JOIN sales USING (Parcel_ID)'''
df_main = pd.read_sql(query, connection)
connection.close()

##### Remove columns that are not of interest

In [ ]:
cols_to_delete = ['Major', 'Minor', 'BldgGradeVar', 'PcntComplete', 'Obsolescence', 'PcntNetCondition', 'Parcel_ID', 
                  'index', 'PropertyType', 'PrincipalUse', 'SaleInstrument', 'AFForestLand', 'AFCurrentUseLand', 'AFNonProfitUse',
                  'AFHistoricProperty', 'SaleReason', 'SaleWarning', 'index']

df_main.drop(cols_to_delete, axis=1, inplace=True)
df_main.dropna(inplace=True)
df_main

##### Limit study to homes with one living unit

As can be seen below, nearly all of the remaining homes have only one living unit. For the sake of consistency, limit the analysis to only single-living unit homes.

In [ ]:
print('Number of homes in analysis, categorized by the number of living units:')
print(df_main.NbrLivingUnits.value_counts())

df_main = df_main.loc[df_main.NbrLivingUnits == 1].copy()
df_main.drop('NbrLivingUnits', axis=1, inplace=True)

## 3. Feature Selection and Creation
Using the above data, categorical inputs will be one-hot encoded such that they can be useful components of the linear regression model. Note that additional variables will be removed along this process. For the sake of keeping this notebook brief, please refer to the exploratory notebooks for further investigation

##### Understand Square Footage metrics and remove unnecessary or redundant data

In [ ]:
# Add two metrics that uncompass the same idea: unfinished area of the home
df_main['SqFtUnfinished'] = df_main['SqFtUnfinFull'] + df_main['SqFtUnfinHalf']

# # Additionally, create a metric for Unfinished basement space, which will be used to analyze finished vs unfinished basements 
df_main['SqFtUnfinBasement'] = df_main.SqFtTotBasement - df_main.SqFtFinBasement

# Remove SquareFootage columns determined to be redundant
sq_ft_cols_to_drop = ['SqFt1stFloor', 'SqFtHalfFloor', 'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFt2ndFloor']
df_main.drop(sq_ft_cols_to_drop, axis=1, inplace=True)

##### Aggregate *Bathroom* data
The original dataset puts bathrooms into three categories: Full, Half, and 3/4. The goal is to combine them into one column, Bathrooms. Method 1 is to count each bathroom the same, regardless of size. Method 2 is to assign them fractional counts based on their naming (1/2, 3/4). The below correlation plot shows that the results is nearly the same, a negligible difference. For the purposes of this analysis, we will use Method 2 so as to not ignore the difference in value that may be added by having a smaller bathroom (e.g. "half bath"). As we can see in the correlation plot below, the two methods are very highly correlated.

In [ ]:
# Define Bathrooms for Method 1 and Method 2, as described above
bathrooms_v1 = df_main[['BathHalfCount', 'Bath3qtrCount', 'BathFullCount']].sum(axis=1)
bathrooms_v2 = df_main['BathHalfCount']/2 + df_main['Bath3qtrCount']*3/4 + df_main['BathFullCount']

# Combine the two methods into one dataframe and visualize their correlation
bathrooms = pd.concat([bathrooms_v1, bathrooms_v2], axis=1)
bathrooms.columns = ['Method 1', 'Method 2']
corr = bathrooms.corr()
sns.heatmap(corr, annot=True)
plt.title('Comparing Two Methods of Calculating the Total Bathrooms')

# Remove columns that are now uncompassed by singluar "Bathrooms" feature
bathroom_cols_to_drop = ['BathHalfCount', 'Bath3qtrCount', 'BathFullCount']
df_main.drop(bathroom_cols_to_drop, axis=1, inplace=True)

# Add "Bathrooms" feature, Method 2, to the main dataframe to continue the analysis
df_main['Bathrooms'] = bathrooms_v2

##### Aggregate *Fireplace* data
Similar to Bathrooms, the original data source breaks down the number of Fireplaces into multiple categories: single story, multi-story, freestanding, and additional. The below code combines them into one final feature, corresponding to the total number of fireplaces. The correlation matrix is added to see how the *Total* correlates with *SalePrice*, as opposed to each individual. 
The results show the *Total Fireplaces* has a higher correlation with *SalePrice* than any of it's components. This shows that we're not ignoring potential important relationships, while also aggregating data that was disaggregated to an extent that it wouldn't be especially helpful as a result of the final analysis.

In [ ]:
# Get all fireplace data in one spot and calculate the total
fireplaces = df_main[['FpSingleStory', 'FpMultiStory', 'FpFreestanding', 'FpAdditional']].copy()
fireplaces['FpTotal'] = fireplaces.sum(axis=1)

# Reorder to make Correlation Matrix easier to understand
fireplaces = fireplaces[['FpTotal', 'FpSingleStory', 'FpMultiStory', 'FpFreestanding', 'FpAdditional']]

# Test correlation of Total with individuals
corr = pd.concat([df_main.SalePrice, fireplaces], axis=1).corr()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

sns.heatmap(corr, annot=True, ax=axes[0]);
axes[0].set_title('Correlation of Fireplace Metrics with SalePrice')

sns.distplot(fireplaces.FpTotal, ax=axes[1])
axes[1].set_title('Total Fireplaces per Home')
axes[1].set_ylabel('Number of Homes (Thousands)')


# Drop columns determined to be less valuable
fp_cols_to_drop = ['FpSingleStory', 'FpMultiStory', 'FpFreestanding', 'FpAdditional']
df_main.drop(fp_cols_to_drop, axis=1, inplace=True)
df_main['Fireplaces'] = fireplaces['FpTotal']

### One-Hot Encoding
For categorical features of interest, convert to one-hot encoded columns
#### Encode HeatSystem

In [ ]:
# Convert encoded values to text values to understand their meaning
keys = get_lookups(108)
convert_to_keys = lambda x: keys[str(int(x))].replace(" ", "").replace('-','') if x!='0' else 'Other'
heating = df_main.HeatSystem.apply(convert_to_keys)
print('Original breakdown:\n{}'.format(heating.value_counts()))

##### Gravity heating is very uncommon. Lump it into the "Other" category

In [ ]:
simplify = lambda x: 'Other' if x in ['Gravity'] else x
heating = heating.apply(simplify)
print('Improved categorization:\n{}'.format(heating.value_counts()))

##### One-hot encode remaining categories
Drop the ForcedAir option. Since it is by far the most common, it will be considered the default option upon which to base alternatives

In [ ]:
# Create dummies
heating_dummies = pd.get_dummies(heating, prefix='Heating', drop_first=False)

# Drop ForcedAir instead of whatever happens to be the first column
heating_dummies.drop('Heating_ForcedAir', axis=1, inplace=True) 

#Adjust df_main accordingly
df_main = pd.concat([df_main, heating_dummies], axis=1)
df_main.drop(['HeatSystem', 'Heating_Other'], axis=1, inplace=True)

#### Encode Condition
Follow process similar to HeatSystem to represent categorical *Condition* feature. Drop the *Average* option since it is naturally a baseline

In [ ]:
# Convert numerical encoding to actual values
keys = get_lookups(83)
convert_to_keys = lambda x: keys[x].replace(' ', '')
condition = df_main.Condition.apply(convert_to_keys)

##### Plot a breakdown to show commonality of categories

In [ ]:
condition_cts = pd.DataFrame(condition.value_counts()).T.reset_index(drop=True)
condition_cts = condition_cts[['Poor', 'Fair', 'Average', 'Good', 'VeryGood']].T #Reorder
plt.bar(condition_cts.index, condition_cts.iloc[:, 0])
plt.title('Commonality of "Condition" Classifications')
plt.ylabel('Number of Homes');

##### One-hot encode Condition

In [ ]:
# Create dummies
condition_dummies = pd.get_dummies(condition, prefix='Condition', drop_first=False)

# Drop Average instead of whatever happens to be the first column
condition_dummies.drop('Condition_Average', axis=1, inplace=True)

#Adjust df_main accordingly
df_main.drop('Condition', axis=1, inplace=True)
df_main = pd.concat([df_main, condition_dummies], axis=1)

#### Encode Porch
By default, the information given about porches is given in square feet. Instead of looking at the impact of porch size, we will investigate whether its presence impacts SalePrice. Encoded as either enclosed, open, both, or neither. The default will be "neither", and as a result we will drop that column once encoded.

In [ ]:
# Create separate DataFrame to temporarily hold Porch data
porches = df_main[['SqFtOpenPorch', 'SqFtEnclosedPorch']].copy()
porches.columns = ['Open', 'Encl']

classify_porches = lambda x: 'Both' if ((x.Open>0)&(x.Encl>0)) else 'Open' if x.Open>0 else 'Closed' if x.Encl>0 else 'None'
porches['Porch'] = porches.apply(classify_porches, axis=1)

##### Visualize the commonality of Porches
As we can see, Open porches are by far the most common type, but it is just as common for a home to not have a porch at all.

In [ ]:
porch_cts = pd.DataFrame(porches.Porch.value_counts())

print('Commonality of Porches\n', porch_cts)
plt.bar(porch_cts.index, porch_cts.Porch)
plt.title('Commonality of "Porch" Types')
plt.ylabel('Number of Homes');

##### One-hot encode Porch types

In [ ]:
porches_dummies = pd.get_dummies(porches['Porch'], prefix='Porch', drop_first=False)
porches_dummies.drop('Porch_None', axis=1, inplace=True)

# Adjust df_main accordingly
df_main = pd.concat([df_main, porches_dummies], axis=1)
df_main.drop(['SqFtOpenPorch', 'SqFtEnclosedPorch'], axis=1, inplace=True)

#### Create binary column for whether a home has been renovated

In [ ]:
encode = lambda x: 1 if x>0 else 0
df_main['Renovated'] = df_main.YrRenovated.apply(encode)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

axes[0].hist(df_main.YrRenovated)
axes[0].set_title("Year Renovated (Zero Indicates Never Renovated)")
axes[0].set_ylabel('Number of Homes');

axes[1].hist(df_main.loc[df_main.YrRenovated>0, 'YrRenovated'])
axes[1].set_title('Year Renovated *For Homes That Have Been Renovated*')
axes[1].set_ylabel('Number of Homes');

##### Add binary indication of renovation (1 = "yes", 0 = "no)

In [ ]:
encode = lambda x: 1 if x>0 else 0
df_main['Renovated'] = df_main.YrRenovated.apply(encode)

#### Create binary column for whether a home's basement is finished

In [ ]:
basement = df_main[['SqFtFinBasement', 'SqFtUnfinBasement']].copy()
basement.columns = ['fin', 'unfin']
classify_basements = lambda x: 'Unfinished' if ((x.fin>0)&(x.unfin>0)) else 'Finished' if x.fin>0 else 'Unfinished' if x.unfin>0 else 'None'
basement['Simplified'] = basement.apply(classify_basements, axis=1)

##### Plot commonality of whether a basement is finished, unfinished, or simply does not exist

In [ ]:
basement_cts = pd.DataFrame(basement.Simplified.value_counts())
plt.bar(basement_cts.index, basement_cts.Simplified)
plt.title('Commonality of Basement Statuses')
plt.ylabel('Number of Homes');

In [ ]:
# One-hot encode
basement_dummies = pd.get_dummies(basement.Simplified, prefix='Basement', drop_first=False)

# Drop 'Unfinished' to set it as the default when investigating the impact of finishing the basement
basement_dummies.drop('Basement_Unfinished', axis=1)

df_main = pd.concat([df_main, basement_dummies], axis=1)
df_main.drop('Basement_None', axis=1, inplace=True)

##### Misc. cleanup

In [ ]:
# Reset index
df_main.reset_index(inplace=True, drop=True)

# Move dependent variable to first column to make future graphs more comprehensible
df_main = pd.concat([df_main['SalePrice'], df_main.drop('SalePrice', axis=1)], axis=1)

## 4. The Model
Below is an investigation of modeling the above-formulated data via inferential linear regression. Only the first and final models are included for the sake of brevity. For a full investigation of the impact of adding specific features, please review the previously referenced Exploratory notebooks.

### Understanding the Data
##### In table format

In [ ]:
df_main

In [ ]:
print('List of columns remaining for analysis:', list(df_main.columns))

##### Graphical: Distribution relationships between features
The below pair-plot shows only continous features. Features with a small number of values don't provide much meaning in this graph. It is also a slow function, which is made slower by including all features.

In [ ]:
# Ignore Seaborn warnings that have previously been determined non-consequential
warnings.filterwarnings("ignore")

# User-defined function that isolates inputs with a minimum number of distinct observations
continuous_features = identify_continuous_features(df_main)

# Distribution Plots for above-derived features
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 10))
for col, ax in zip(continuous_features.columns, axes.flatten()):
    sns.distplot(continuous_features[col], ax=ax)
    ax.set_title(col)
    ax.set_xlabel('')
    plt.subplots_adjust(wspace=.5, hspace=.5)
fig.suptitle('Distribution of Continuous Features');

#### Graphical: Correlations

##### Heat map for continuous features (as defined above)

In [ ]:
corr = continuous_features.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool)) # Formula taken from Flatiron School study group material
fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corr, mask=mask, ax=ax, annot=True);

##### Heat map for features with few distinct inputs (i.e. those note above), plotted with SalePrice

In [ ]:
df_discrete = pd.concat([df_main.SalePrice, df_main.drop(continuous_features.columns, axis=1)], axis=1)

corr = df_discrete.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool)) # Formula taken from Flatiron School study group material
fig, ax = plt.subplots(figsize=(18, 14))
sns.heatmap(corr, mask=mask, ax=ax, annot=True);

##### List correlations of SalePrice and input features

In [ ]:
correlations = df_main.corr().stack().reset_index()
correlations = correlations.loc[(correlations.level_0 == 'SalePrice') & (correlations.level_1 != 'SalePrice')]
correlations.columns = ['Dependent', 'Feature', 'Correlation (abs)']
correlations['Correlation (abs)'] = correlations['Correlation (abs)'].abs()
correlations.sort_values(by='Correlation (abs)', ascending=False, inplace=True, ignore_index=True)
print(correlations)

### The Baseline Model
After reviewing the above table, we see that Livable Square Footage is the highest predictor or Sale Price. This isn't surprising. Let's create the simple model and check the assumptions of Linear Regression. Let's also create a dataframe to store the model performance metrics so we can compare them as we go.

In [ ]:
performance_metrics_cols = ['Y', 'X', 'Linearity p-value', 'Jarque-Bera (JB) metric', 'JB p-value', 'Lagrange multiplier', 'Lagrange multiplier p-value', 'F-score', 'F-score p-value', 'Average VIF', 'R^2 (Adj.)']
performance_metrics = pd.DataFrame(columns = performance_metrics_cols)

In [ ]:
inputs = ['SqFtTotLiving']
output = 'SalePrice'

model, df_model = produce_model(df_main, inputs, output)
results = check_assumptions(model, df_model, output, verbose=True, feature_to_plot='SqFtTotLiving')
performance_metrics = performance_metrics.append(results)

print(model.summary())
performance_metrics

##### Analysis of above model
As we can see, it's not linear. In fact, only one of the necessary fundamental assumptions is met. That said, it is impossible to not meet given one input feature. That is of course Multicollinearity. We see that in the Variance Inflation Factor (VIF) producing an NA. All other statistics are especially bad:
1. **Linearity: failed.** The inputs are not linear, as described by the linear rainbow test. A p-value greater than 0.05 would indicate linearity. This model produces a p-value of 0.000004. Not good.
2. **Normality of Residuals: failed.** The model residuals are not normally distributed. This is seen in the QQ-plot and the Jarque-Bera (JB) metric and its associated p-value. Although the metric would ideally be less than 5 and the p-value would be greater than 0.05, we instead have a metric equal to 13168580 and a p-value equal to effectively 0. Not good.
3. **Homoscedacity: failed.** The residuals are not homoscedastic. This is visualized in the scatter plot above and quantified by the F-score and its associated p-value. Lower F-scores are better, and the p-value should be above 0.05. Instead, the F-score in 439 and the associated p-value is 2.18e-96. Not good.
4. **Multi-collinearity: passed by default.** this is inherently not an issue as discussed above solely because there is only one input, thus making collinearity impossible. 

### Final Model

We see that there are extensive improvements to be made to the above model. For a full walk-through of the discovery process of getting to this final model, please review the Exploratory notebooks. In brief, here are some improvements made to the model:
1. SalePrice is log-transformed. We can see in the distribution plot above that there is a severe skew to the SalePrice data. This is nearly entirely reconciled by log-transforming the output, as is done in the final model.
2. SqFtTotLiving is also log-transformed for a very similar reason. It improves linearity drastically. 
3. Binary feature corresponding to whether or not the basement has been finished was added.
4. One-hot encoded features corresponding to the porch are added, measuring the impact of an Open porch, and Enclosed porch, or Both.
5. One-hot encoded features corresponding to the type of heating used in the home are added. 

As a result, all performance metrics have improved.

##### Log-transform SqFtLivingArea and SalePrice

In [ ]:
log_transform = lambda x: np.log(x)    
df_main['SalePrice_log'] = df_main.SalePrice.apply(log_transform)
df_main['SqFtTotLiving_log'] = df_main.SqFtTotLiving.apply(log_transform)

In [ ]:
0.009/0.05

##### (move this down)

##### Analysis of the final model
There have in fact been substantial improvements in all metrics necessary to meet the fundamental assumptions of linear regression. Two of four have reached the necessary threshold. The other two have not, but have got closer.
1. **Linearity: passed.** The linearity assumption is met because the p-value associated with the linear rainbow test is 0.37, far above the predetermined 0.05 alpha threshold required. This improvement came largely from log-transforming the SalePrice and the SqFtLivingArea, however, there were plenty of features that were tested in the model development process that were excluded in part because they pummeled the linearity p-value.
2. **Normality of Residuals: failed.** The normality of residuals, measured by the JB metric and p-value, have a ways to go. The JB stat decreased from 1.3e7 to 6300. This is a huge improvement, but the associated p-value still rounds to zero, meaning it is far from checking the box.
3. **Homoscedacity: failed but *almost* passed.** The plot above shows only the residuals for SqFtTotLiving, but there are drastic improvements in the F-scoe, Langrange multiplier, and their associated p-values. For example, the p-value for the score increased from 2e-96 to 0.009. That leaves the p-value less than one order of magnitude away from meeting the requirement. Although it isn't success, it's a huge step.
4. **Multi-collinearity: passed** The multi-collinearity assumption, as measured by VIF metrics is met. The average VIF is roughly 1.3. Given that anything below 5 is considered passable, this one is a pass. 



In [ ]:
# Define model inputs and output
inputs = ['SqFtTotLiving_log', 'Basement_Finished', 'Porch_Open', 'Porch_Closed', 'Porch_Both', 'Heating_ElecBB', 'Heating_FloorWall', 'Heating_HeatPump', 'Heating_HotWater','Heating_Radiant']
output = 'SalePrice_log'

# Run user-defined functions to create model and subsequently check the assumptions of linear regression
model, df_model = produce_model(df_main, inputs, output)
results = check_assumptions(model, df_model, output, verbose=True, feature_to_plot='SqFtTotLiving_log')
performance_metrics = performance_metrics.append(results)

print(model.summary())
performance_metrics

## 5. Quantify The Findings

#### Quantify the import of coefficients given log-scaled output: create DataFrame with results
##### Create column to indicate whether a feature was log-transformed

In [ ]:
results = pd.DataFrame(model.params).reset_index()
results.columns = ['attribute', 'coeff']
results['log_transformed?'] = ['_log' in x for x in results.attribute]

##### For feature that **were not** log transformed: calculate the "Percent change in SalePrice per *UNIT* input increase"

In [ ]:
# Reverse log calculation
exp_func = lambda x: np.round(10**x, 2)-1
results['% change in SalePrice per *unit* input increase'] = results['coeff'].apply(exp_func)

# Set intercept equal to NA as it cannot be interpretted this way
results.iloc[0, -1] = 'NA'

# Set features that *were* log transformed equal to NA. They need their own column
results.loc[results['log_transformed?']==True, '% change in SalePrice per *unit* input increase'] = 'NA'

##### For feature that **were** log transformed: calculate the "Percent change in SalePrice per *PERCENT* input increase"

In [ ]:
# Store converted values
logged_inputs_converted = []

# Loop over all features
for row in range(results.shape[0]):
    
    # Enter statement if a feature was log transformed
    if results.iloc[row]['log_transformed?'] == True:
        
        # 1.01 corresponds to a 1% increase
        x = 1.01**results.iloc[row]['coeff']
        logged_inputs_converted.append(x-1)
    
    else:
        # This is not a valid metric for inputs that weren't transformed. Set to NA
        logged_inputs_converted.append('NA')
results['% change in SalePrice per *percent* input increase']  = logged_inputs_converted

In [ ]:
results

### Findings

#### Enclose the porch
Take note of the **Porch_** metrics in the *Results* table. Note that these are one-hot encoded, with the default being no porch at all. To put it into context, homes with *Open* porches tend to sell for roughly 8% more than homes without a porch at all. However, that number jumps to 27% for homes with *Enclosed* porches, meaning that homes with *enclosed* porches sell for nearly 20% more than homes with *open* porches. It looks like it's time to enclose that porch of yours!

#### Finish the basement
Refer to the **Basement_Finished** row in the *Results* table. This is one-hot encoded to indicate whether or not a basement is finished or unfinished. Note that this excludes homes with no basement at all. As we can see, homes with finished basements typically sell for 18% more than homes with unfinished basements. Maybe it's worth the time and money to get it finished up. 

#### Choosing a heating system
Refer to the **Heating_** rows in the *Results* table. These are one-hot encoded values, with the default being Forced Air, given that it is by far the most common. It is used in over 75% of the 18,000 homes analyzed. Here, the results are unrealistically extreme. Instead of looking at the degree to which a heating system can impact home price, let's instead view it as a *trend*. The heating sources in the most expensive sales are Radiant and Hot Water. Heat pump and Floor-Wall are also correlated with a higher price than Forced Air. Coming in last, Electric Baseboard heating is associated with the lowest sale prices of homes. 

### Consider an add-on
Amongst the strongest predictors of sale price is the **Livable Square Feet** in a home. This is not surprising. The relationship is striking: for every 10% increase in square footage, there is a 7% increase in home value. That's substantial. It is also a possible explanation as to why finishing the basement can have such a positive impact. 